# ライブラリのインポート

In [ ]:
import os
import time
import json
import datetime
import zoneinfo

import requests

from dotenv import load_dotenv, find_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin

from azure.identity.aio import DefaultAzureCredential

from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread
from semantic_kernel.contents import ChatMessageContent, FunctionCallContent, FunctionResultContent

# 環境変数の取得

In [77]:
load_dotenv(override=True)

PROJECT_ENDPOINT=os.getenv("PROJECT_ENDPOINT")
AZURE_DEPLOYMENT_NAME=os.getenv("AZURE_DEPLOYMENT_NAME")
AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY=os.getenv("AZURE_OPENAI_API_KEY")

PROJECT_ENDPOINT=os.getenv("PROJECT_ENDPOINT")
FOUNDRY_CUSTOM_FUNCTIONS_AGENT_ID=os.getenv("FOUNDRY_CUSTOM_FUNCTIONS_AGENT_ID")
FOUNDRY_CODE_INTERPRETER_AGENT_ID=os.getenv("FOUNDRY_CODE_INTERPRETER_AGENT_ID")
FOUNDRY_MCP_AGENT_ID=os.getenv("FOUNDRY_MCP_AGENT_ID")
FOUNDRY_FILE_SEARCH_AGENT_ID=os.getenv("FOUNDRY_FILE_SEARCH_AGENT_ID")

# ユーティリティ関数

In [70]:
import datetime
import json
from semantic_kernel.contents.function_call_content import FunctionCallContent
from semantic_kernel.contents.function_result_content import FunctionResultContent
from semantic_kernel.contents.text_content import TextContent


async def print_thread_message_details(thread: str):
    """
    スレッドのメッセージ詳細を表示します。

    Args:
        thread (str): スレッドのインスタンス
    """
    async for message in thread.get_messages():
        print("-----")

        for item in message.items:
            if isinstance(item, FunctionCallContent):
                print(f"[Function Calling] by {message.ai_model_id}")
                print(f" - Function Name : {item.name}")
                print(f" - Arguments     : {item.arguments}")

            elif isinstance(item, FunctionResultContent):
                print(f"[Function Result]")
                # 文字列のデコード変換
                if isinstance(item.result, str):
                    try:
                        decoded = json.loads(item.result)
                        print(f" - Result        : {decoded}") # デコード成功時は変換後の値を表示
                    except json.JSONDecodeError:
                        print(f" - Result        : {item.result}")  # デコード失敗時はそのまま
                else:
                    print(f" - Result        : {item.result}")

            elif isinstance(item, TextContent):
                if message.name:
                    print(f"[Agent Response] from {message.ai_model_id}")
                else:
                    print("[User Message]")
                print(f" - Content       : {item.text}")

            else:
                print(f"[Unknown Item Type] ({type(item).__name__})")
                print(f" - Raw Item      : {item}")


def log_with_timestamp(message: str) -> None:
    """
    現在時刻付きでメッセージを標準出力にログとして表示します。

    Args:
        message (str): 出力するログメッセージ。
    """
    timestamp = datetime.datetime.now().strftime("%H:%M:%S.%f")[:-3]
    print(f"[{timestamp}] {message}")


# プラグインの作成

In [ ]:
# MCPのエンドポイントURL
MCP_URL = "http://127.0.0.1:8000/mcp/"


mcp_plugin = MCPStreamableHttpPlugin(
    name="mcp_plugin",
    url=MCP_URL
)

await mcp_plugin.connect()

# クライアントの初期化

In [105]:
# Chat Completion API クライアントの初期化
azure_completion_service  = AzureChatCompletion(
    service_id="azure_completion_agent",
    deployment_name=AZURE_DEPLOYMENT_NAME,
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY
)

In [106]:
# Chat Completion Agent クライアントの初期化
agent = ChatCompletionAgent(
    service=azure_completion_service,
    name="MCPAgent",
    instructions=(
        "あなたはECサイトの分析エージェントです。"
        "PostgreSQLの各種マスター、注文、ユーザー情報と、CosmosDBに格納されたレビュー分析データを、mcp_pluginとして利用できます。"
    ),
    plugins=[mcp_plugin],
)

## スレッドの作成

In [114]:
# Thread の作成
thread = ChatHistoryAgentThread()
print(f"Created Thread. THREAD_ID: {thread.id}")


Created Thread. THREAD_ID: thread_d2e082f275aa42aeb200480f2e207cc3


## レスポンスを取得

In [115]:
response = await agent.get_response(
    messages=["2024年12月は何がどれくらい購入されましたか？"],
    thread=thread,
)

print(response)

2024年12月に購入された主な商品と売上金額（上位）は以下の通りです。

- Surface Pro：1,440,000円
- Surface Laptop：170,000円
- Surface Go：90,000円
- Modern Webcam：39,000円
- Office：38,000円
- Xbox Game Pass：1,500円

このデータは売上金額順（購入金額の多い順）となっています。他の商品や詳細な数量情報が必要な場合はご指定ください。


In [116]:
await print_thread_message_details(thread)

-----
[User Message]
 - Content       : 2024年12月は何がどれくらい購入されましたか？
-----
[Function Calling] by gpt-4.1
 - Function Name : mcp_plugin-get_sales_by_product
 - Arguments     : {"start_date":"2024-12-01","end_date":"2024-12-31"}
-----
[Function Result]
 - Result        : [TextContent(inner_content=TextContent(type='text', text='[{"product_id": 1, "product_name": "Surface Pro", "total_sales": 1440000}, {"product_id": 2, "product_name": "Surface Laptop", "total_sales": 170000}, {"product_id": 3, "product_name": "Surface Go", "total_sales": 90000}, {"product_id": 8, "product_name": "Modern Webcam", "total_sales": 39000}, {"product_id": 11, "product_name": "Office", "total_sales": 38000}, {"product_id": 15, "product_name": "Xbox Game Pass", "total_sales": 1500}]', annotations=None, meta=None), ai_model_id=None, metadata={}, content_type='text', text='[{"product_id": 1, "product_name": "Surface Pro", "total_sales": 1440000}, {"product_id": 2, "product_name": "Surface Laptop", "total_sales": 1700